In [19]:
# # basic module
# import pandas as pd
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
# %matplotlib inline
# import os
# import shutil
# import glob
# import time
# import pickle

# plt.style.use('seaborn-dark')
# %config InlineBackend.figure_format = 'retina'
# pd.options.display.max_rows = 20
# pd.options.display.max_columns = 20

# plt.rcParams["figure.figsize"] = (14,4)
# plt.rcParams['lines.linewidth'] = 2
# plt.rcParams['lines.color'] = 'r'
# plt.rcParams['axes.grid'] = False


# # image module
# import cv2
# from pydicom import dcmread
# import pylibjpeg
# from PIL import Image
# import pydicom as dcm
# from pydicom import dcmread
# from pydicom.data import get_testdata_file



# # dataset
# # # df = pd.read_csv('dataset.csv',index_col=0)

# # # image_path
# # d1p = '/home/ncp/workspace/data/DL/d1/'
# # d1 = os.listdir(d1p)
# # d1.sort()

# # d2p = '/home/ncp/workspace/data/DL/d2/'
# # d2 = os.listdir(d2p)
# # d2.sort()

# # d3p = '/home/ncp/workspace/data/DL/d3/'
# # d3 = os.listdir(d3p)
# # d3.sort()

# # d4p = '/home/ncp/workspace/data/DL/d4/'
# # d4 = os.listdir(d4p)
# # d4.sort()

# # d5p = '/home/ncp/workspace/data/DL/d5/'
# # d5 = os.listdir(d5p)
# # d5.sort()

# # d6p = '/home/ncp/workspace/data/DL/d6/'
# # d6 = os.listdir(d6p)
# # d6.sort()

# # d9p = '/home/ncp/workspace/data/DL/d9/'
# # d9 = os.listdir(d9p)
# # d9.sort()

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

from torch import optim
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os
import torchvision
from torchvision import utils


from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
from torchsummary import summary
import numpy as np
import time
import copy
import random
import tqdm
import math

device = torch.device('cpu')
# import gc

# gc.collect()

# torch.cuda.empty_cache()

In [39]:
# img = Image.open('/home/ncp/workspace/data/train/04.신생아호흡곤란증후군/A1.0-2개월/H05_00019_02/org/H05_00019_02.png')
transf = Compose([Resize((384, 384)), ToTensor(),Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ])
dataset = torchvision.datasets.ImageFolder(root='/home/ncp/workspace/data/DL',transform=transf)
# dataset = torchvision.datasets.ImageFolder(root='/home/ncp/workspace/data/DL')
# dataset = transf(dataset).unsqueeze(0)
train_set, val_set, test_set = torch.utils.data.random_split(dataset,[3000,800,200])

In [40]:
dataset[0][0].shape

torch.Size([3, 384, 384])

In [41]:
classes = dataset.classes
classes

['d1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd9']

In [42]:
train_dl = DataLoader(train_set,batch_size=64,shuffle = True,num_workers = 4)
val_dl = DataLoader(val_set,batch_size=64,shuffle = True,num_workers = 4)
test_dl = DataLoader(test_set,batch_size=64,shuffle = True,num_workers = 4)

In [43]:
# def show(img, y=None):
#     npimg = img.numpy()
#     npimg_tr = np.transpose(npimg, (1, 2, 0))
#     plt.imshow(npimg_tr)

#     if y is not None:
#         plt.title('labels:' + str(y))

# np.random.seed()
# torch.manual_seed(0)

# grid_size=4
# rnd_ind = np.random.randint(0, len(train_set), grid_size)

# x_grid = [train_set[i][0] for i in rnd_ind]
# y_grid = [val_set[i][1] for i in rnd_ind]

# x_grid = utils.make_grid(x_grid, nrow=grid_size, padding=2)
# plt.figure(figsize=(10,10))
# show(x_grid, y_grid)

In [44]:
# def imshow(img):
#     img = img /2 + 0.5
#     np_img = img.numpy()
#     plt.imshow(np.transpose(np_img, (1,2,0)))
#     print(np_img.shape)
#     print((np.transpose(np_img, (1,2,0))).shape)

In [45]:
# print(images.shape)
# imshow(torchvision.utils.make_grid(images,nrow=4))
# print(images.shape)
# print((torchvision.utils.make_grid(images)).shape)
# print("".join("%5s " % classes[labels[j]] for j in range(16)))

In [46]:
# class PatchEmbedding(nn.Module):
#     def __init__(self, in_channels: int = 4, patch_size: int = 16, emb_size: int = 768):
#         self.patch_size = patch_size
#         super().__init__()
#         self.projection = nn.Sequential(
#             # using a conv layer instead of a linear one -> performance gains
#             nn.Conv2d(in_channels, emb_size, kernel_size=patch_size, stride=patch_size),
#             Rearrange('b e (h) (w) -> b (h w) e'),
#         )
        
#         self.cls_token = nn.Parameter(torch.randn(1,1, emb_size))
        
#     def forward(self, x: Tensor) -> Tensor:
#         b, _, _, _ = x.shape
#         x = self.projection(x)
#         cls_tokens = repeat(self.cls_token, '() n e -> b n e', b=b)
#         # prepend the cls token to the input
#         x = torch.cat([cls_tokens, x], dim=1)
#         return x
    
# PatchEmbedding()(x).shape

In [47]:
class PatchEmbedding(nn.Module):
    def __init__(self, in_channels: int = 3, patch_size: int = 16, emb_size: int = 768, img_size: int = 224):
        self.patch_size = patch_size
        super().__init__()
        self.projection = nn.Sequential(
            # using a conv layer instead of a linear one -> performance gains
            nn.Conv2d(in_channels, emb_size, kernel_size=patch_size, stride=patch_size),
            Rearrange('b e (h) (w) -> b (h w) e'),
        )
        self.cls_token = nn.Parameter(torch.randn(1,1, emb_size))
        self.positions = nn.Parameter(torch.randn((img_size // patch_size) **2 + 1, emb_size))

        
    def forward(self, x: Tensor) -> Tensor:
        b, _, _, _ = x.shape
        x = self.projection(x)
        cls_tokens = repeat(self.cls_token, '() n e -> b n e', b=b)
        # prepend the cls token to the input
        x = torch.cat([cls_tokens, x], dim=1)
        # add position embedding
        x += self.positions
        return x
    
# PatchEmbedding()(x).shape

In [48]:
# Check PatchEmbedding
x = torch.randn(16, 3, 224, 224).to(device)
patch_embedding = PatchEmbedding().to(device)
patch_output = patch_embedding(x)
print('[batch, 1+num of patches, emb_size] = ', patch_output.shape)

[batch, 1+num of patches, emb_size] =  torch.Size([16, 197, 768])


In [49]:
class MultiHeadAttention(nn.Module):
    def __init__(self, emb_size=768, num_heads=8, dropout=0):
        super().__init__()
        self.emb_size = emb_size
        self.num_heads = num_heads
        self.keys = nn.Linear(emb_size, emb_size)
        self.queries = nn.Linear(emb_size, emb_size)
        self.values = nn.Linear(emb_size, emb_size)
        self.att_drop = nn.Dropout(dropout)
        self.projection = nn.Linear(emb_size, emb_size)

    def forward(self, x, mask=None):
        # split keys, queries and values in num_heads
        queries = rearrange(self.queries(x), 'b n (h d) -> b h n d', h=self.num_heads) # b, 197, 728 -> b, 8, 197, 91
        keys = rearrange(self.keys(x), 'b n (h d) -> b h n d', h=self.num_heads)
        values = rearrange(self.values(x), 'b n (h d) -> b h n d', h=self.num_heads)
        # sum up over the last axis, b,h,197,197
        energy = torch.einsum('bhqd, bhkd -> bhqk', queries, keys) # batch, num_head, query_len, key_len
        
        if mask is not None:
            fill_value = torch.finfo(torch.float32).min
            energy.mask_fill(~mask, fill_value)
        
        scaling = self.emb_size ** (1/2)
        att = F.softmax(energy, dim=-1) / scaling
        att = self.att_drop(att)
        # sum up over the third axis
        out = torch.einsum('bhal, bhlv -> bhav', att, values) # 197x91
        out = rearrange(out, 'b h n d -> b n (h d)')
        out = self.projection(out)
        return out

In [50]:
# class MultiHeadAttention(nn.Module):
#     def __init__(self, emb_size: int = 768, num_heads: int = 8, dropout: float = 0):
#         super().__init__()
#         self.emb_size = emb_size
#         self.num_heads = num_heads
#         # fuse the queries, keys and values in one matrix
#         self.qkv = nn.Linear(emb_size, emb_size * 3)
#         self.att_drop = nn.Dropout(dropout)
#         self.projection = nn.Linear(emb_size, emb_size)
        
#     def forward(self, x : Tensor, mask: Tensor = None) -> Tensor:
#         # split keys, queries and values in num_heads
#         qkv = rearrange(self.qkv(x), "b n (h d qkv) -> (qkv) b h n d", h=self.num_heads, qkv=3)
#         queries, keys, values = qkv[0], qkv[1], qkv[2]
#         # sum up over the last axis
#         energy = torch.einsum('bhqd, bhkd -> bhqk', queries, keys) # batch, num_heads, query_len, key_len
#         if mask is not None:
#             fill_value = torch.finfo(torch.float32).min
#             energy.mask_fill(~mask, fill_value)
            
#         scaling = self.emb_size ** (1/2)
#         att = F.softmax(energy, dim=-1) / scaling
#         att = self.att_drop(att)
#         # sum up over the third axis
#         out = torch.einsum('bhal, bhlv -> bhav ', att, values)
#         out = rearrange(out, "b h n d -> b n (h d)")
#         out = self.projection(out)
#         return out
    
# patches_embedded = PatchEmbedding()(x)
# MultiHeadAttention()(patches_embedded).shape

In [51]:
MHA = MultiHeadAttention()
MHA_output = MHA(patch_output)
print(MHA_output.shape)

torch.Size([16, 197, 768])


In [52]:
class ResidualAdd(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn
        
    def forward(self, x, **kwargs):
        res = x
        x = self.fn(x, **kwargs)
        x += res
        return x

In [53]:
class FeedForwardBlock(nn.Sequential):
    def __init__(self, emb_size: int, expansion: int = 4, drop_p: float = 0.):
        super().__init__(
            nn.Linear(emb_size, expansion * emb_size),
            nn.GELU(),
            nn.Dropout(drop_p),
            nn.Linear(expansion * emb_size, emb_size),
        )

In [54]:
x = torch.randn(16,1,128).to(device)
model = FeedForwardBlock(128).to(device)
output = model(x)
print(output.shape)

torch.Size([16, 1, 128])


In [55]:
class TransformerEncoderBlock(nn.Sequential):
    def __init__(self,
                 emb_size: int = 768,
                 drop_p: float = 0.,
                 forward_expansion: int = 4,
                 forward_drop_p: float = 0.,
                 ** kwargs):
        super().__init__(
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                MultiHeadAttention(emb_size, **kwargs),
                nn.Dropout(drop_p)
            )),
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                FeedForwardBlock(
                    emb_size, expansion=forward_expansion, drop_p=forward_drop_p),
                nn.Dropout(drop_p)
            )
            ))

In [56]:
# Check TransformerEncoderBlock
model = TransformerEncoderBlock().to(device)
output = model(patch_output).to(device)
print(output.shape)

torch.Size([16, 197, 768])


In [57]:
class TransformerEncoder(nn.Sequential):
    def __init__(self, depth: int = 12, **kwargs):
        super().__init__(*[TransformerEncoderBlock(**kwargs) for _ in range(depth)])

In [58]:
model = TransformerEncoder().to(device)
output = model(patch_output)
print(output.shape)

torch.Size([16, 197, 768])


In [59]:
class ClassificationHead(nn.Sequential):
    def __init__(self, emb_size: int = 768, n_classes: int = 1000):
        super().__init__(
            Reduce('b n e -> b e', reduction='mean'),
            nn.LayerNorm(emb_size), 
            nn.Linear(emb_size, n_classes))

In [60]:
# check
x = torch.randn(16, 1, 768).to(device)
model = ClassificationHead().to(device)
output = model(x)
print(output.shape)

torch.Size([16, 1000])


In [61]:
class ViT(nn.Sequential):
    def __init__(self,     
                in_channels: int = 3,
                patch_size: int = 16,
                emb_size: int = 768,
                img_size: int = 224,
                depth: int = 12,
                n_classes: int = 1000,
                **kwargs):
        super().__init__(
            PatchEmbedding(in_channels, patch_size, emb_size, img_size),
            TransformerEncoder(depth, emb_size=emb_size, **kwargs),
            ClassificationHead(emb_size, n_classes)
        )
        

In [62]:
# x = torch.randn(16,3,224,224).to(device)
# model = ViT().to(device)
# output = model(x)
# print(output.shape)

In [63]:
# import gc

# gc.collect()

# torch.cuda.empty_cache()

In [64]:
model = ViT().to(device)
summary(model, (3,224,224), device=device.type)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 768, 14, 14]         590,592
         Rearrange-2             [-1, 196, 768]               0
    PatchEmbedding-3             [-1, 197, 768]               0
         LayerNorm-4             [-1, 197, 768]           1,536
            Linear-5             [-1, 197, 768]         590,592
            Linear-6             [-1, 197, 768]         590,592
            Linear-7             [-1, 197, 768]         590,592
           Dropout-8          [-1, 8, 197, 197]               0
            Linear-9             [-1, 197, 768]         590,592
MultiHeadAttention-10             [-1, 197, 768]               0
          Dropout-11             [-1, 197, 768]               0
      ResidualAdd-12             [-1, 197, 768]               0
        LayerNorm-13             [-1, 197, 768]           1,536
           Linear-14            [-1, 1

In [65]:
loss_func = nn.CrossEntropyLoss(reduction='sum')
opt = optim.Adam(model.parameters(), lr=0.01)

from torch.optim.lr_scheduler import ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(opt, mode='min', factor=0.1, patience=10)

In [66]:
def get_lr(opt):
    for param_group in opt.param_groups:
        return param_group['lr']

In [67]:
def metric_batch(output, target):
    pred = output.argmax(1, keepdim=True)
    corrects = pred.eq(target.view_as(pred)).sum().item()
    return corrects


# calculate the loss per mini-batch
def loss_batch(loss_func, output, target, opt=None):
    loss_b = loss_func(output, target)
    metric_b = metric_batch(output, target)

    if opt is not None:
        opt.zero_grad()
        loss_b.backward()
        opt.step()
    
    return loss_b.item(), metric_b


# calculate the loss per epochs
def loss_epoch(model, loss_func, dataset_dl, sanity_check=False, opt=None):
    running_loss = 0.0
    running_metric = 0.0
    len_data = len(dataset_dl.dataset)

    for xb, yb in dataset_dl:
        xb = xb.to(device)
        yb = yb.to(device)
        output = model(xb)

        loss_b, metric_b = loss_batch(loss_func, output, yb, opt)

        running_loss += loss_b
        
        if metric_b is not None:
            running_metric += metric_b

        if sanity_check is True:
            break

    loss = running_loss / len_data
    metric = running_metric / len_data
    return loss, metric

In [68]:
def train_val(model, params):
    num_epochs=params['num_epochs']
    loss_func=params['loss_func']
    opt=params['optimizer']
    train_dl=params['train_dl']
    val_dl=params['val_dl']
    sanity_check=params['sanity_check']
    lr_scheduler=params['lr_scheduler']
    path2weights=params['path2weights']

    loss_history = {'train': [], 'val': []}
    metric_history = {'train': [], 'val': []}

    best_loss = float('inf')
    best_model_wts = copy.deepcopy(model.state_dict())
    start_time = time.time()

    for epoch in range(num_epochs):
        current_lr = get_lr(opt)
        print('Epoch {}/{}, current lr= {}'.format(epoch, num_epochs-1, current_lr))

        model.train()
        train_loss, train_metric = loss_epoch(model, loss_func, train_dl, sanity_check, opt)
        loss_history['train'].append(train_loss)
        metric_history['train'].append(train_metric)

        model.eval()
        with torch.no_grad():
            val_loss, val_metric = loss_epoch(model, loss_func, val_dl, sanity_check)
        loss_history['val'].append(val_loss)
        metric_history['val'].append(val_metric)

        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), path2weights)
            print('Copied best model weights!')

        lr_scheduler.step(val_loss)
        if current_lr != get_lr(opt):
            print('Loading best model weights!')
            model.load_state_dict(best_model_wts)

        print('train loss: %.6f, val loss: %.6f, accuracy: %.2f, time: %.4f min' %(train_loss, val_loss, 100*val_metric, (time.time()-start_time)/60))
        print('-'*10)

    model.load_state_dict(best_model_wts)
    return model, loss_history, metric_history

In [69]:
params_train = {
    'num_epochs':10,
    'optimizer':opt,
    'loss_func':loss_func,
    'train_dl':train_dl,
    'val_dl':val_dl,
    'sanity_check':False,
    'lr_scheduler':lr_scheduler,
    'path2weights':'./models/weights.pt',
}

# check the directory to save weights.pt
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSerror:
        print('Error')
createFolder('./models')

In [70]:
model, loss_hist, metric_hist = train_val(model, params_train)

Epoch 0/9, current lr= 0.01


RuntimeError: DataLoader worker (pid(s) 6856) exited unexpectedly

In [ ]:
num_epochs = params_train['num_epochs']

# Plot train-val loss
plt.title('Train-Val Loss')
plt.plot(range(1, num_epochs+1), loss_hist['train'], label='train')
plt.plot(range(1, num_epochs+1), loss_hist['val'], label='val')
plt.ylabel('Loss')
plt.xlabel('Training Epochs')
plt.legend()
plt.show()

# plot train-val accuracy
plt.title('Train-Val Accuracy')
plt.plot(range(1, num_epochs+1), metric_hist['train'], label='train')
plt.plot(range(1, num_epochs+1), metric_hist['val'], label='val')
plt.ylabel('Accuracy')
plt.xlabel('Training Epochs')
plt.legend()
plt.show()